# MYSQL lab 9

Create a Python connection with SQL database and retrieve the results of the following queries as dataframes:

1. How many distinct (different) actors' last names are there?
2. Add an additional column day_type with values 'weekend' and 'workday' depending on the rental day of the week.
3. Get all films with ARMAGEDDON in the title.
4. Get 10 the longest films.
5. How many films include Behind the Scenes content?
6.  Which kind of movies (rating) have a mean duration of more than two hours?
7.  Rank films by length (filter out the rows that have nulls or 0s in length column). In your output, only select the columns title, length, and the rank.

In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass

In [2]:

# Read database password from user and establish database connection
password = getpass.getpass('Enter database password: ')
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'

engine = create_engine(connection_string)


Enter database password: ········


In [3]:

# 1. How many distinct (different) actors' last names are there?
query = "SELECT count(DISTINCT last_name) AS total_last_names FROM actor"
data = pd.read_sql_query(query, engine)
data.head()


,total_last_names
0,121


In [4]:

# 2. Add an additional column day_type with values 'weekend' and 'workday'
#    depending on the rental day of the week.
query = """SELECT *,
           CASE
             WHEN weekday(rental_date) < 6 then 'workday'
             ELSE 'weekend'
           END AS 'day_type'
           FROM rental;"""
data = pd.read_sql_query(query, engine)
data.head()


,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update,day_type
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53,workday
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53,workday
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53,workday
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53,workday
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53,workday


In [5]:

# 3. Get all films with ARMAGEDDON in the title.

query = "SELECT film_id,title FROM film WHERE title LIKE '%%ARMAGEDDON%%'"
data = pd.read_sql_query(query, engine)
data.head()


,film_id,title
0,39,ARMAGEDDON LOST
1,507,LADYBUGS ARMAGEDDON
2,571,METAL ARMAGEDDON
3,598,MOSQUITO ARMAGEDDON
4,838,STAGECOACH ARMAGEDDON


In [6]:

# 4. Get 10 the longest films.
query = "SELECT * FROM film ORDER BY length DESC LIMIT 10"
data = pd.read_sql_query(query, engine)
data.head()


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,817,SOLDIERS EVOLUTION,A Lacklusture Panorama of a A Shark And a Pion...,2006,1,None,7,4.99,185,27.99,R,"Trailers,Commentaries,Deleted Scenes,Behind th...",2006-02-15 05:03:42
1,349,GANGS PRIDE,A Taut Character Study of a Woman And a A Shar...,2006,1,None,4,2.99,185,27.99,PG-13,Behind the Scenes,2006-02-15 05:03:42
2,609,MUSCLE BRIGHT,A Stunning Panorama of a Sumo Wrestler And a H...,2006,1,None,7,2.99,185,23.99,G,Deleted Scenes,2006-02-15 05:03:42
3,872,SWEET BROTHERHOOD,A Unbelieveable Epistle of a Sumo Wrestler And...,2006,1,None,3,2.99,185,27.99,R,Deleted Scenes,2006-02-15 05:03:42
4,141,CHICAGO NORTH,A Fateful Yarn of a Mad Cow And a Waitress who...,2006,1,None,6,4.99,185,11.99,PG-13,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42


In [7]:

# 5. How many films include Behind the Scenes content?
query = "SELECT count(*) FROM film WHERE special_features LIKE '%%Behind the Scenes%%'"
data = pd.read_sql_query(query, engine)
data.head()


,count(*)
0,538


In [8]:

# 6.  Which kind of movies (rating) have a mean duration of more than two hours?
query = """SELECT rating, round(avg(length), 2) as average_length
               FROM film
               GROUP BY rating
               HAVING average_length > 120;"""
data = pd.read_sql_query(query, engine)
data.head()


,rating,average_length
0,PG-13,120.44


In [9]:

# 7. Rank films by length (filter out the rows that have nulls or 0s in length column).
# In your output, only select the columns title, length, and the rank.
query = """SELECT *, RANK() OVER (ORDER BY length) AS ranking
               FROM film
               WHERE length > 0 AND NOT ISNULL(length);"""
data = pd.read_sql_query(query, engine)
data.head()


,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,ranking
0,15,ALIEN CENTER,A Brilliant Drama of a Cat And a Mad Scientist...,2006,1,None,5,2.99,46,10.99,NC-17,"Trailers,Commentaries,Behind the Scenes",2006-02-15 05:03:42,1
1,469,IRON MOON,A Fast-Paced Documentary of a Mad Cow And a Bo...,2006,1,None,7,4.99,46,27.99,PG,"Commentaries,Behind the Scenes",2006-02-15 05:03:42,1
2,504,KWAI HOMEWARD,A Amazing Drama of a Car And a Squirrel who mu...,2006,1,None,5,0.99,46,25.99,PG-13,"Trailers,Commentaries",2006-02-15 05:03:42,1
3,505,LABYRINTH LEAGUE,A Awe-Inspiring Saga of a Composer And a Frisb...,2006,1,None,6,2.99,46,24.99,PG-13,"Commentaries,Behind the Scenes",2006-02-15 05:03:42,1
4,730,RIDGEMONT SUBMARINE,A Unbelieveable Drama of a Waitress And a Comp...,2006,1,None,3,0.99,46,28.99,PG-13,"Commentaries,Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,1
